In [1]:
from html import escape
from decimal import Decimal

In [18]:
def html_escape(arg):
    return escape(str(arg))

def html_int(a):
    return f'{a}(<i>{str(hex(a))}</i>)'

def html_real(a):
    return f'{a:.2f}'

def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

def html_list(l):
    items = (f'<li>{htmlize(item)}</li>' for item in l)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_dict(d):
    items = (f'<li>{html_escape(k)}={htmlize(v)}</li>' for k, v in d.items())
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [19]:
def htmlize(arg):
    if isinstance(arg, int):
        return html_int(arg)
    elif isinstance(arg, float) or isinstance(arg, Decimal):
        return html_real(arg)
    elif isinstance(arg, str):
        return html_str(arg)
    elif isinstance(arg, list) or isinstance(arg, tuple):
        return html_list(arg)
    elif isinstance(arg, dict):
        return html_dict(arg)
    else:
        return html_escape(arg)
    

In [21]:
print(htmlize(["""Python 
rocs! 0 < 1
""", (10, 20, 30), 100]))

<ul>
<li>Python <br/>
rocs! 0 &lt; 1<br/>
</li>
<li><ul>
<li>10(<i>0xa</i>)</li>
<li>20(<i>0x14</i>)</li>
<li>30(<i>0x1e</i>)</li>
</ul></li>
<li>100(<i>0x64</i>)</li>
</ul>


## Second iteration

In [8]:
def htmlize(arg):
    
    registry = {
        object: html_escape,
        int: html_int,
        float: html_real,
        Decimal: html_real,
        str: html_str,
        list: html_list,
        tuple: html_list,
        set: html_list,
        dict: html_dict,
    }

    fn = registry.get(type(arg), registry[object])
    return fn(arg)

def html_escape(arg):
    return escape(str(arg))

def html_int(a):
    return f'{a}(<i>{str(hex(a))}</i>)'

def html_real(a):
    return f'{a:.2f}'

def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

def html_list(l):
    items = (f'<li>{htmlize(item)}</li>' for item in l)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_dict(d):
    items = (f'<li>{html_escape(k)}={htmlize(v)}</li>' for k, v in d.items())
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [9]:
print(htmlize(["""Python 
rocs! 0 < 1
""", (10, 20, 30), 100]))

<ul>
<li>Python <br/>
rocs! 0 &lt; 1<br/>
</li>
<li><ul>
<li>10(<i>0xa</i>)</li>
<li>20(<i>0x14</i>)</li>
<li>30(<i>0x1e</i>)</li>
</ul></li>
<li>100(<i>0x64</i>)</li>
</ul>


## Third iteration

In [18]:
def single_dispatch(fn):
    registry = {}

    registry[object] = fn
    registry[int] = lambda a: f'{a}(<i>{str(hex(a))}</i>)'
    registry[str] = lambda s: escape(s).replace('\n', '<br/>\n')

    def decorated(arg):
        return registry.get(type(arg), registry[object])(arg)
    
    def register(type_):
        def inner(fn):
            registry[type_] = fn
            return fn
        return inner
    
    def dispatch(type_):
        return registry.get(type_, registry[object])
    
    decorated.register = register
    decorated.dispatch = dispatch
    return decorated


@single_dispatch
def htmlize(a):
    return escape(str(a))


htmlize('1 < 100')

'1 &lt; 100'

In [15]:
@htmlize.register(int)
def html_int(a):
    return f'{a}(<i>{str(hex(a))}</i>)'

htmlize(100)

'100(<i>0x64</i>)'

In [17]:
@htmlize.register(tuple)
@htmlize.register(list)
def html_sequence(l):
    items = (f'<li>{htmlize(item)}</li>' for item in l)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

print(htmlize(["""Python 
rocs! 0 < 1
""", (10, 20, 30), 100]))

<ul>
<li>Python <br/>
rocs! 0 &lt; 1<br/>
</li>
<li>(10, 20, 30)</li>
<li>100(<i>0x64</i>)</li>
</ul>


In [21]:
@htmlize.register(Integral)
def html_integral_number(a):
    return f'{a}(<i>{str(hex(a))}</i>)'

In [22]:
htmlize(10)

'10(<i>0xa</i>)'

In [24]:
@htmlize.register(Sequence)
def html_sequence(l):
    items = (f'<li>{htmlize(item)}</li>' for item in l)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

### Using singledispatch from functools

In [8]:
from functools import singledispatch
from numbers import Integral
from collections.abc import Sequence
from html import escape
from decimal import Decimal

@singledispatch
def htmlize(a):
    return escape(str(a))

@htmlize.register(Integral)
def html_integral_number(a):
    return f'{a}(<i>{str(hex(a))}</i>)'

@htmlize.register(Sequence)
def html_sequence(l):
    items = (f'<li>{htmlize(item)}</li>' for item in l)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

@htmlize.register(str)
def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

@htmlize.register(tuple)
def html_tuple(t):
    items = (escape(str(item)) for item in t)
    return f'{", ".join(items)}'

In [6]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>,
              numbers.Integral: <function __main__.html_integral_number(a)>,
              collections.abc.Sequence: <function __main__.html_sequence(l)>})

In [4]:
htmlize.dispatch(int)

<function __main__.html_integral_number(a)>